In [6]:
#render geo coor gps to image

import cv2
import numpy as np
import pandas as pd
import os
import pickle
import sys
sys.path.append('..')
from coordinate_transform import wgs_to_gcj02_to_mercator
from tqdm import tqdm


# 示例GPS点数据，假设数据包含 'longitude' 和 'latitude' 列
gps_path = "datasets/trips_0_24"
iter = tqdm(os.listdir(gps_path))
save_path = 'datasets/trips_0_24_image'
if not os.path.isdir(save_path):
    os.makedirs(save_path)

# tile's coordinate bounds
bounds_path = "datasets/0_24_gps.txt"
with open(bounds_path, 'r') as f:
    tile_bounds = eval(f.read())

for gps_name in iter:
    iter.set_description(gps_name)
    gps_df = pd.read_csv(os.path.join(gps_path, gps_name), header=None)
    gps_df.columns = ['id', 'lat', 'lon', 'time']
    # 创建一个空白的1024x1024二值图像
    img_size = 1024
    binary_image = np.zeros((img_size, img_size), dtype=np.uint8)

    # 映射GPS点到图像坐标系
    for index, row in gps_df.iterrows():
        # convert wgs84 to web mercator under gcj02
        row['lon'], row['lat'] = wgs_to_gcj02_to_mercator(row['lon'], row['lat'])

        if (tile_bounds['west'] <= row['lon'] <= tile_bounds['east'] and
                tile_bounds['south'] <= row['lat'] <= tile_bounds['north']):

            # 计算点在图像上的位置
            x = int((row['lon'] - tile_bounds['west']) / (tile_bounds['east'] - tile_bounds['west']) * (img_size - 1))
            y = int((tile_bounds['north'] - row['lat']) / (tile_bounds['north'] - tile_bounds['south']) * (img_size - 1))

            # 设置对应位置为白色（255）
            binary_image[y, x] = 255

    # 保存图像
    cv2.imwrite(f'{save_path}/{gps_name[:-4]}.png', binary_image)


trip_1109.txt: 100%|██████████| 283/283 [00:11<00:00, 25.37it/s]


In [1]:
# re arrange the traj data by time

import cv2
import numpy as np
import pandas as pd
import os
import pickle
import sys
sys.path.append('..')
from coordinate_transform import wgs_to_gcj02_to_mercator
from tqdm import tqdm

gps_path = "datasets/trips_0_24"
iter = tqdm(os.listdir(gps_path))
save_path = 'datasets/trips_0_24_sorted'
if not os.path.isdir(save_path):
    os.makedirs(save_path)

for file_name in iter:
    iter.set_description(file_name)
    gps_df = pd.read_csv(os.path.join(gps_path, file_name), header=None)
    gps_df.columns = ['id', 'lat', 'lon', 'time']
    gps_df = gps_df.sort_values(by='time')
    gps_df.to_csv(f'{save_path}/{file_name}', index=False, header=False)

print("Done!")

trip_1109.txt: 100%|██████████| 283/283 [00:01<00:00, 183.64it/s]


Done!
